How State is Updated: Recommended Methods

1. The Easy Way: output_key (for Agent Text Responses)

In [ ]:
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.genai.types import Content, Part

# Define agent with output_key
greeting_agent = LlmAgent(
    name="Greeter",
    model="gemini-2.0-flash", # Use a valid model
    instruction="Generate a short, friendly greeting.",
    output_key="last_greeting" # Save response to state['last_greeting']
)

# --- Setup Runner and Session ---
app_name, user_id, session_id = "state_app", "user1", "session1"
session_service = InMemorySessionService()
runner = Runner(
    agent=greeting_agent,
    app_name=app_name,
    session_service=session_service
)
session = await session_service.create_session(app_name=app_name, 
                                    user_id=user_id, 
                                    session_id=session_id)
print(f"Initial state: {session.state}")
print(f"greeting_agent: {greeting_agent}")
print(f"session service {session_service} and type {type(session_service)}")
print(f"session {session} and type{type(session)}")

# --- Run the Agent ---
# Runner handles calling append_event, which uses the output_key
# to automatically create the state_delta.
user_message = Content(parts=[Part(text="Hello")])
for event in runner.run(user_id=user_id, 
                        session_id=session_id, 
                        new_message=user_message):
    if event.is_final_response():
      print(f"Agent responded.") # Response text is also in event.content

# --- Check Updated State ---
updated_session = await session_service.get_session(app_name=app_name, user_id=user_id, session_id=session_id)
print(f"State after agent run: {updated_session.state}")
# Expected output might include: {'last_greeting': 'Hello there! How can I help you today?'}

Initial state: {}
greeting_agent: name='Greeter' description='' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='Generate a short, friendly greeting.' global_instruction='' tools=[] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key='last_greeting' planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None
session service <google.adk.sessions.in_memory_session_service.InMemorySessionService object at 0x000001E85DF1E410> and type <class 'google.adk.sessions.in_memory_session_service.InMemorySessionService'>
session id='session1' app_name='state_app' user_id='user1' state={} events=[] last_update_time=1750232625.7010972 and type<class 'google.adk.sessions.session.Session'>
Agent responded.
State after agent 

2. The Standard Way: EventActions.state_delta (for Complex Updates)

In [15]:
from google.adk.sessions import InMemorySessionService, Session
from google.adk.events import Event, EventActions
from google.genai.types import Part, Content
import time

# --- Setup ---
session_service = InMemorySessionService()
app_name, user_id, session_id = "state_app_manual", "user2", "session2"
session = await session_service.create_session(
    app_name=app_name,
    user_id=user_id,
    session_id=session_id,
    state={"user:login_count": 0, "task_status": "idle"}
)
# print(f"session service: {session_service}")
# print(f"session: {session}")
# print(f"Initial session state: {session.state}")

# --- Define State Changes ---
current_time = time.time()
state_changes = {
    "task_status": "active",              # Update session state
    "user:login_count": session.state.get("user:login_count", 0) + 1, # Update user state
    "user:last_login_ts": current_time,   # Add user state
    "temp:validation_needed": True        # Add temporary state (will be discarded)
}

# --- Create Event with Actions ---
actions_with_update = EventActions(state_delta=state_changes)
# This event might represent an internal system action, not just an agent response
system_event = Event(
    invocation_id="inv_login_update",
    author="system", # Or 'agent', 'tool' etc.
    actions=actions_with_update,
    timestamp=current_time
    # content might be None or represent the action taken
)

print("actions_with_update: ", actions_with_update)
print("system_event: ", system_event)

# --- Append the Event (This updates the state) ---
await session_service.append_event(session, system_event)
print("`append_event` called with explicit state delta.")

# --- Check Updated State ---
updated_session = await session_service.get_session(app_name=app_name,
                                            user_id=user_id, 
                                            session_id=session_id)
print(f"State after event: {updated_session.state}")
# Expected: {'user:login_count': 1, 'task_status': 'active', 'user:last_login_ts': <timestamp>}
# Note: 'temp:validation_needed' is NOT present.

actions_with_update:  skip_summarization=None state_delta={'task_status': 'active', 'user:login_count': 1, 'user:last_login_ts': 1750230355.5376706, 'temp:validation_needed': True} artifact_delta={} transfer_to_agent=None escalate=None requested_auth_configs={}
system_event:  content=None grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None invocation_id='inv_login_update' author='system' actions=EventActions(skip_summarization=None, state_delta={'task_status': 'active', 'user:login_count': 1, 'user:last_login_ts': 1750230355.5376706, 'temp:validation_needed': True}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}) long_running_tool_ids=None branch=None id='EkA1tOr7' timestamp=1750230355.5376706
`append_event` called with explicit state delta.
State after event: {'user:login_count': 1, 'task_status': 'active', 'user:last_login_ts': 1750230355.5376706}


3. Via CallbackContext or ToolContext (Recommended for Callbacks and Tools)

In [ ]:
# In an agent callback or tool function
from google.adk.agents.callback_context import CallbackContext # or ToolContext

def my_callback_or_tool_function(context: CallbackContext, # Or ToolContext
                                 # ... other parameters ...
                                ):
    # Update existing state
    count = context.state.get("user_action_count", 0)
    context.state["user_action_count"] = count + 1

    # Add new state
    context.state["temp:last_operation_status"] = "success"

    # State changes are automatically part of the event's state_delta
    # ... rest of callback/tool logic ...